In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.nn.init as init
import torch.autograd as autograd
from torch.nn.parameter import Parameter


In [ ]:
## The data loader, you should import it correctly by adding the functions file to the path, but I could not be bothered to do that right now.
#%%
import pandas as pd

# Define a custom function to parse the CSV file
def custom_csv_parser(file_path, Training=True):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    # Initialize dictionaries to store different sections
    data = {}
    for line in lines:
        line = line.strip()  # Remove leading and trailing whitespace
        if not line:  # Skip empty lines
            continue
        
        if line.startswith('Meal_size') or line.startswith('Steady_insulin') or line.startswith('Bolus'):
            key, value, _ = line.split(',')
            data[key] = float(value)
        elif line.startswith('D1') or line.startswith('D2'):
            parts = line.split(',')
            key = parts[0]
            values = [float(x) for x in parts[2:] if x]  # Filter out empty strings
            data[key] = values
        elif line.startswith('I_sc') or line.startswith('I_p') or line.startswith('I_eff') or line.startswith('G') or line.startswith('G_sc'):
            parts = line.split(',')
            key = parts[0]
            values = [float(x) for x in parts[1:] if x]  # Filter out empty strings
            data[key] = values
        elif line.startswith('tau1') or line.startswith('tau2') or line.startswith('Ci') or line.startswith('p2') or line.startswith('Si') or line.startswith('GEZI') or line.startswith('EGP0') or line.startswith('Vg') or line.startswith('taum') or line.startswith('tausc'):
            if line.startswith('Si') and not Training:
                key, value, _ = line.split(',')
                data[key] = float(value)
    if Training:
        print('Training data loaded successfully!')
    else:
        print('Test data loaded successfully!')
    return data

# file_path = '../Patient.csv'
# # Load the training data
# train_data = custom_csv_parser(file_path = '../Patient.csv')
# # Load the test data
# test_data = custom_csv_parser(file_path = '../Patient.csv', Training=False)


In [ ]:
class PINN(nn.module):

    def __init__(self):
        

    def MVP():


        # T

        D1 = Parameter()
        D2 = Parameter()

        I_sc = Parameter()
        I_p = Parameter()
        I_eff = Parameter()

        G = Parameter()
        G_sc = Parameter()

        # Patient Variables

        tau_1 = 49 #; % min
        tau_2 = 47 #; % min
        C_I = 20.1 #; % dL/min
        p_2 = 0.0106 #; % 1/min
        GEZI = 0.0022 #; % 1/min
        EGP_0 = 1.33 #; % mg/dl/min
        V_G = 253 #; % dL
        tau_m = 47 #; % min
        tau_sc = 5 #; % min

        # The secret unknown parameter
        # S_I = 0.0081 #; % (dL/mU)/min
        S_I_guess = 0.01
        S_I = Parameter(S_I_guess)
    

        ## Define the gradients, this is easier since we are working with odes

        D1_t = D1.grad()
        D2_t = D2.grad()

        I_sc_t = I_sc.grad()
        I_p_t = I_p.grad()
        I_eff_t = I_eff.grad()

        G_t = G.grad()
        G_sc_t = G_sc.grad()

        ## Define the differential equations

        # Meal equations
        eq_M1 = D1_t - (d - D1/tau_m)
        eq_M2 =  D2_t - ( (D1-D2) / tau_m )

        # Insulin equations
        eq_I1 = I_sc_t - (u/(tau_1*C_I) - I_sc/tau_1)
        eq_I2 = I_p_t - ( (I_sc-I_p)/tau_2 )
        eq_I3 = I_eff_t - (-p_2*I_eff + p_2*S_I*I_p)

        # Glucose equations
        eq_G1 = G_t - (( GEZI + I_eff) * G + EGP_0 + 1000*D2/(V_G*tau_m) )



        